In [ ]:
import re
import csv
import pandas as pd
from bs4 import BeautifulSoup
import spacy
from top2vec import Top2Vec

In [2]:
df=pd.read_csv("C_78DM8fG6E.csv",skipinitialspace=True)

In [3]:
def clean_text(text):
    # Remove HTML tags
    soup = BeautifulSoup(text, 'html.parser')
    text = soup.get_text()
    
    # Remove special ASCII symbols
    text = re.sub(r'[^\x00-\x7F]+', '', text)

    # Remove asterisks around each letter in the phrase and words attached to @
    text = re.sub(r'\*|@\w+|@|\?|!', '', text)
    
    return text.strip()

In [4]:
df['clean_comment'] = df['Comment'].apply(clean_text)

C:\Users\s570350\AppData\Local\Temp\ipykernel_6968\1728796713.py:3: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, 'html.parser')


In [5]:
# Removing stop words.
# Function to remove stopwords and names from a sentence
def remove_stopwords_and_names(sentence):
    # Load English language model
    nlp = spacy.load("en_core_web_sm")
    # Process the sentence with spaCy
    doc = nlp(sentence)
    # Remove stop words from the sentence
    filtered_sentence = " ".join(token.text for token in doc if not token.is_stop and token.ent_type_ == '')
    return filtered_sentence

In [ ]:
# Remove stopwords from the 'text_column'
df['clean_comment'] = df['clean_comment'].apply(remove_stopwords_and_names)

In [ ]:
def filter_sentences(text):
    sentences = text.split('.')
    filtered_sentences = [sentence for sentence in sentences if len(sentence.split()) >= 5]
    return '. '.join(filtered_sentences)


# Apply the function to the 'clean_comment' column
df['clean_comment'] = df['clean_comment'].apply(filter_sentences)

In [ ]:
# Function to filter out rows with empty 'clean_comment' values
def filter_empty_comments(row):
    return row['clean_comment'] != '' or row['clean_comment'] != None

# Apply the function to filter out rows with empty 'clean_comment' values
df = df[df.apply(filter_empty_comments, axis=1)]

In [ ]:
# Save the cleaned DataFrame to a new CSV file
df["clean_comment"].to_csv("C_78DM8fG6E_2.csv", index=False)

In [ ]:
import spacy
spacy.load('en_core_web_sm')
from spacy.lang.en import English
parser = English()
def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        elif token.orth_.startswith('@'):
            lda_tokens.append('SCREEN_NAME')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

In [ ]:
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet as wn
def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma
    
from nltk.stem.wordnet import WordNetLemmatizer
def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

In [ ]:
nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))

In [ ]:
def prepare_text_for_lda(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 4]
    tokens = [token for token in tokens if token not in en_stop]
    tokens = [get_lemma(token) for token in tokens]
    return tokens

In [ ]:
token_list = []
document = df['clean_comment'].tolist()
for line in document:
        tokens = prepare_text_for_lda(line)
        token_list += tokens

In [ ]:
df["tokens"] = df["clean_comment"].apply(prepare_text_for_lda)

In [ ]:
def get_topic_dict(text):
    dict_ = {}
    text = text.split("+")
    for i in range(len(text)):
        text[i] = text[i].strip()
        temp = text[i].split("*")
        if temp[0] != "":
            dict_[temp[1][1:-1]] = float(temp[0])
    
    return dict_

In [ ]:
import gensim
from gensim import corpora
def get_topics(tokens):
    if len(tokens) != 0:
        dictionary = corpora.Dictionary([tokens])
        corpus = [dictionary.doc2bow(tokens)]
        NUM_TOPICS = 1
        ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
        ldamodel.save('model5.gensim')
        topics = ldamodel.print_topics(num_words=len(tokens))
        return get_topic_dict(topics[0][1])
    else:
        return {}

In [ ]:
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)


In [ ]:
df = df[df['topics'] != {}]
df

In [ ]:
for i in df['topics']:
    print(i)

In [ ]:
# Save the cleaned DataFrame to a new CSV file
df.to_csv("C_78DM8fG6E_2.csv", index=False)

In [ ]:
df.head()